In [ ]:
!pip install torch
!pip install torchani

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

# Adding ProtoSyn.jl package

In [ ]:
using Pkg
Pkg.add(url="https://github.com/JosePereiraUA/ONNX.jl", rev = "ops-fix")
Pkg.add(url="https://github.com/sergio-santos-group/ProtoSyn.jl", rev = "dev")

In [ ]:
using ProtoSyn

# Projeto do módulo 2 de LCVC

## 1. Perform a mutation (to any aminoacid you’d like, other than Proline – PRO/P) on any residue you’d like. Showcase the mutation performed.

Possivéis escolhas de aminoácidos


In [ ]:
ProtoSyn.three_2_one

In [ ]:
Set(values(ProtoSyn.three_2_one))

Escolher o residuo e aminoácido:

In [ ]:
seleResID = 48; 
strResID = rid"48";
seleAmino = "M";

Carregar a proteína original.

In [ ]:
pose = ProtoSyn.Peptides.load("mol1.pdb")

Aplicar a mutação:

In [ ]:
ProtoSyn.Peptides.mutate!(pose, pose.graph[1, seleResID], ProtoSyn.Peptides.grammar, [seleAmino])

Gravar a proteína final:

In [ ]:
ProtoSyn.write(pose, "mol1_mutated.pdb")

---

## 2. Set-up and run a Monte Carlo simulation to find the best rotamer for the newly mutated aminoacid. Use the Dunbrack 2011 rotamer library. The Monte-Carlo simulation should run for 100 steps, using a constant temperature of 0.01. Evaluate each conformation attempt in the Monte Carlo with an energy function using TorchANI as the single energy function component. (4/10)

### a. Print and visualize (i.e.: show) each frame of the simulation, using a Callback, highlighting the conformational changes being made. (1/10)


Função de energia (TorchANI ensemble):

In [ ]:
energy_function = ProtoSyn.Calculators.EnergyFunction()
torchani = ProtoSyn.Calculators.TorchANI.get_default_torchani_ensemble()
push!(energy_function, torchani)

**Define the RotamerMutator.**
( the Dunbrack 2011 rotamer library), the `p_mut` probability of mutation (1.0), the `n_first` most likely rotamers to consider (100), the targeted residue (as a selection, rid"48") and finally whether to define random _phi_ and _psi_ dihedral angles in the terminals (set to false, doesn't have any effect for residue 48).

In [ ]:
rotamer_library = ProtoSyn.Peptides.load_dunbrack()
rotamer_mutator = ProtoSyn.Peptides.Mutators.RotamerMutator(rotamer_library, 1.0, 100, strResID, false)

Definir callback:

In [ ]:
callback = ProtoSyn.Common.default_energy_step_frame_callback(1, "mol1_simulation.pdb")

Definir o termostato com temperatura, T = 0.01:

In [ ]:
thermostat = ProtoSyn.Drivers.get_constant_temperature(0.01)

Criar o Monte Carlo:

In [ ]:
monte_carlo = ProtoSyn.Drivers.MonteCarlo(
  energy_function,
  rotamer_mutator,
  callback,
  100, #steps
  thermostat)

Carregar a molécula mutada anteriormente para aplicação do método de Monte Carlo:

In [ ]:
pose = ProtoSyn.Peptides.load("mol1_mutated.pdb")

Guardar o estado inicial:

In [ ]:
ProtoSyn.write(pose, "mol1_simulation.pdb")

Simulação:

In [ ]:
monte_carlo(pose)

## 4. Perform the same simulation, on the same residue, but applying different initial mutations (i.e.: mutate to different target aminoacids). Are there any noticeable differences both in energy and structure? (2/10)

### -> repetir mas mudar o "R" para outro aminoacido

In [ ]:
a